In [16]:
import pandas as pd
import ast
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# ✅ Configuration
DEBUG_MODE = False
SAMPLE_SIZE = 1000
TOP_K = 2500
SAVE_PATH = "/content/drive/MyDrive/hmm_model/"

# ✅ Load your cleaned dataset
print("📥 Loading dataset...")
df = pd.read_csv(f"{SAVE_PATH}cleaned_stackoverflow.csv")

# ✅ Debug sampling
if DEBUG_MODE:
    df = df.sample(n=SAMPLE_SIZE, random_state=42)
    print(f"🧪 DEBUG MODE ENABLED: Using only {SAMPLE_SIZE} samples")
else:
    print("🚀 FULL TRAINING MODE ENABLED")

# ✅ Parse tag strings
def safe_parse(tag_str):
    try:
        return ast.literal_eval(tag_str) if isinstance(tag_str, str) else tag_str
    except:
        return []

df['tags'] = df['tags'].apply(safe_parse)

# ✅ Count and select top-K tags
print("🔢 Counting tags...")
all_tags = sum(df['tags'].tolist(), [])
tag_counts = Counter(all_tags)
top_tags = set(tag for tag, _ in tag_counts.most_common(TOP_K))
print(f"✅ Top {TOP_K} tags selected.")

# ✅ Filter samples that contain at least one top tag
filtered_texts, filtered_tags = [], []
for text, tag_list in zip(df['text'], df['tags']):
    tags = [tag for tag in tag_list if tag in top_tags]
    if tags:
        filtered_texts.append(text)
        filtered_tags.append(tags)

print(f"✅ Filtered dataset: {len(filtered_texts)} samples retained.")

# ✅ MultiLabel binarization
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(filtered_tags)
print(f"🏷️ Label matrix shape: {labels.shape} (samples x tags)")

# ✅ Split into train/val (90:10)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    filtered_texts, labels, test_size=0.1, random_state=42
)

print(f"📦 Train samples: {len(train_texts)}")
print(f"🧪 Val samples: {len(val_texts)}")
print(f"🔠 Unique tags: {len(mlb.classes_)}")

# ✅ Save the splits for reference
train_df = pd.DataFrame({'text': train_texts, 'tags': mlb.inverse_transform(train_labels)})
val_df = pd.DataFrame({'text': val_texts, 'tags': mlb.inverse_transform(val_labels)})

# Convert tags to comma-separated strings
train_df['tags'] = train_df['tags'].apply(lambda tag_list: ",".join(tag_list))
val_df['tags'] = val_df['tags'].apply(lambda tag_list: ",".join(tag_list))

# Save to Drive
train_df.to_csv(f"{SAVE_PATH}train_split.csv", index=False)
val_df.to_csv(f"{SAVE_PATH}val_split.csv", index=False)
print(f"💾 Train and validation splits saved to: {SAVE_PATH}")


📥 Loading dataset...
🚀 FULL TRAINING MODE ENABLED
🔢 Counting tags...
✅ Top 2500 tags selected.
✅ Filtered dataset: 48627 samples retained.
🏷️ Label matrix shape: (48627, 2500) (samples x tags)
📦 Train samples: 43764
🧪 Val samples: 4863
🔠 Unique tags: 2500
💾 Train and validation splits saved to: /content/drive/MyDrive/hmm_model/


,text,tags
0,prevent scroll into nothing while draging I us...,"css,reactjs,scrollview"
1,PDO MySQL Connection close - unset vs null I'v...,"pdo,php"
2,Gorm Timestamps in Golang type Base struct {\n...,go
3,What is the difference between Spyder and Jupy...,"jupyter,python,spyder"
4,Load nested settings from yaml file using pyda...,"pydantic,pydantic-v2,python"


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer
import numpy as np

# ✅ Auto-detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)
if device.type == "cuda":
    print("🚀 GPU:", torch.cuda.get_device_name(0))
else:
    print("🧱 CPU mode")

# ✅ Load tokenizer once
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# ✅ Dataset class
class TagDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.from_numpy(np.array(labels)).float()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'labels': self.labels[idx]
        }

# ✅ Create dataset instances
train_dataset = TagDataset(train_texts, train_labels)
val_dataset = TagDataset(val_texts, val_labels)

# ✅ Use batch size = 16 (safe for T4 GPU)
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ✅ Print one sample for sanity check
sample = train_dataset[0]
print(f"\n🔍 Sample input_ids shape: {sample['input_ids'].shape}")
print(f"🔍 Sample label shape: {sample['labels'].shape}")


✅ Using device: cuda
🚀 GPU: Tesla T4

🔍 Sample input_ids shape: torch.Size([128])
🔍 Sample label shape: torch.Size([2500])


In [18]:
import torch
import torch.nn as nn
from transformers import DistilBertModel

# ✅ Automatically use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Define the model class
class MiniTagTransformer(nn.Module):
    def __init__(self, num_tags):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_tags)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        return self.classifier(cls_output)

# ✅ Initialize and move model to device
model = MiniTagTransformer(num_tags=len(mlb.classes_)).to(device)
print(f"✅ Model initialized with {len(mlb.classes_)} output tags.")
print(f"📦 Model is on device: {next(model.parameters()).device}")

✅ Model initialized with 2500 output tags.
📦 Model is on device: cuda:0


In [19]:
from tqdm import tqdm
import torch.optim as optim
import pickle

# ✅ Ensure model and data are on correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ✅ Loss & Optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

# ✅ Training settings
best_val_loss = float('inf')
patience = 2
patience_counter = 0
EPOCHS = 10 # 🧪 Debug/test mode — increase later

for epoch in range(EPOCHS):
    print(f"\n🔁 Epoch {epoch + 1}/{EPOCHS}")
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_loader, desc="🧠 Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # ✅ Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="🧪 Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            loss = loss_fn(logits, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"📉 Train Loss: {avg_train_loss:.4f} | 📊 Val Loss: {avg_val_loss:.4f}")

    # ✅ Checkpointing & Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), '/content/drive/MyDrive/hmm_model/trained_model.pt')
        with open('/content/drive/MyDrive/hmm_model/mlb.pkl', 'wb') as f:
            pickle.dump(mlb, f)
        print("💾 Best model saved!")
    else:
        patience_counter += 1
        print(f"⏳ Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("🛑 Early stopping triggered.")
            break



🔁 Epoch 1/10


🧪 Validating: 100%|██████████| 304/304 [00:51<00:00,  5.88it/s]


📉 Train Loss: 0.0497 | 📊 Val Loss: 0.0079
💾 Best model saved!

🔁 Epoch 2/10


🧪 Validating: 100%|██████████| 304/304 [00:51<00:00,  5.88it/s]


📉 Train Loss: 0.0071 | 📊 Val Loss: 0.0066
💾 Best model saved!

🔁 Epoch 3/10


...

📉 Train Loss: 0.0028 | 📊 Val Loss: 0.0033
💾 Best model saved!

🔁 Epoch 9/10


🧪 Validating: 100%|██████████| 304/304 [00:51<00:00,  5.86it/s]


📉 Train Loss: 0.0025 | 📊 Val Loss: 0.0032
💾 Best model saved!

🔁 Epoch 10/10


🧪 Validating: 100%|██████████| 304/304 [00:51<00:00,  5.85it/s]


📉 Train Loss: 0.0022 | 📊 Val Loss: 0.0031
💾 Best model saved!


In [23]:
import torch
import numpy as np
import pickle
from transformers import DistilBertTokenizer, DistilBertModel
from torch import nn

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load tokenizer and label encoder
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
with open('/content/drive/MyDrive/hmm_model/mlb.pkl', 'rb') as f:
    mlb = pickle.load(f)

# ✅ Define model architecture
class MiniTagTransformer(nn.Module):
    def __init__(self, num_tags):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_tags)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        return self.classifier(cls_output)

# ✅ Load trained model
model = MiniTagTransformer(num_tags=len(mlb.classes_))
model.load_state_dict(torch.load('/content/drive/MyDrive/hmm_model/trained_model.pt', map_location=device))
model.to(device)
model.eval()

# ✅ Tag prediction function with fallback & debug
def predict_tags(text, threshold=0.15, show_top_k=5, fallback=True):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        logits = model(inputs["input_ids"], inputs["attention_mask"])
        probs = torch.sigmoid(logits).squeeze().cpu().numpy()

    # 🔍 Debug: print top K tag probabilities
    print(f"\n🔍 Top {show_top_k} tag probabilities:")
    top_probs = sorted(enumerate(probs), key=lambda x: x[1], reverse=True)[:show_top_k]
    for i, p in top_probs:
        print(f"{mlb.classes_[i]}: {p:.4f}")

    # ✅ Select tags above threshold
    predicted_indices = np.where(probs >= threshold)[0]
    tags = [mlb.classes_[i] for i in predicted_indices]

    # 🛟 Fallback to top-k if no confident tags
    if fallback and not tags:
        print("⚠️ No confident tags, returning top-K tags as fallback.")
        tags = [mlb.classes_[i] for i, _ in top_probs]

    return tags

example_question_1 = """
How to resolve CORS error when making a fetch request in JavaScript?

I'm working on a front-end application using plain JavaScript, and I'm trying to make a fetch call to an external API (https://api.example.com/data). However, I'm getting the following error in the console:

Access to fetch at 'https://api.example.com/data' from origin 'http://localhost:3000' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource.

I've tried setting mode: 'no-cors' in the fetch request, but it doesn't help. I'm not using any backend, just a simple HTML file served locally. Is there any way to fix this issue on the front-end or do I need to configure something on the server?

Here's my fetch code:

fetch('https://api.example.com/data', {
  method: 'GET',
  headers: {
    'Content-Type': 'application/json'
  }
})
.then(response => response.json())
.then(data => console.log(data))
.catch(error => console.error(error));
"""

example_question_2 = """
How can I rewrite a subquery into a JOIN for better performance in SQL?

I have this SQL query using a subquery that works, but it's quite slow:

SELECT name FROM employees WHERE department_id IN (
  SELECT id FROM departments WHERE location = 'New York'
);

I've read that using JOINs might be more efficient. How can I convert this to use an INNER JOIN instead of a subquery?
"""

example_question_3 = """
What is the best way to manage state in Flutter for a simple counter app?

I'm new to Flutter and trying to build a simple counter app. I want to update the counter value and reflect the changes in the UI. I've heard about Provider, Riverpod, and Bloc — but I’m not sure which one to use for a small app.

Do I need a package like Provider or can I just use setState inside a StatefulWidget?
"""

test_questions = [example_question_1, example_question_2, example_question_3]

for idx, q in enumerate(test_questions, 1):
    print(f"\n🧪 Test Question {idx}:")
    predicted_tags = predict_tags(q, threshold = 0.05, show_top_k=5)
    print("🔮 Predicted Tags:", predicted_tags)





🧪 Test Question 1:

🔍 Top 5 tag probabilities:
javascript: 0.9951
cors: 0.6005
reactjs: 0.5653
json: 0.2206
ajax: 0.1992
🔮 Predicted Tags: ['ajax', 'cors', 'express', 'fetch-api', 'html', 'javascript', 'jquery', 'json', 'node.js', 'post', 'postman', 'reactjs', 'rest']

🧪 Test Question 2:

🔍 Top 5 tag probabilities:
sql: 0.9765
sql-server: 0.4457
postgresql: 0.1132
t-sql: 0.0633
sql-server-2008: 0.0212
🔮 Predicted Tags: ['postgresql', 'sql', 'sql-server', 't-sql']

🧪 Test Question 3:

🔍 Top 5 tag probabilities:
flutter: 0.9981
dart: 0.3016
bloc: 0.1213
flutter-bloc: 0.1106
riverpod: 0.0691
🔮 Predicted Tags: ['bloc', 'dart', 'flutter', 'flutter-bloc', 'flutter-dependencies', 'riverpod']


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np
import time

# ⏱️ Start timing
start = time.time()

# 📦 Initialize binary predictions and true labels
y_true = []
y_pred = []

print(f"🧪 Starting evaluation on {len(val_texts)} validation samples...")

# 🔁 Loop through val_texts with progress bar
for idx, text in tqdm(enumerate(val_texts), total=len(val_texts), desc="🧠 Evaluating"):
    # 🔮 Predict tags (using your original predict_tags function)
    predicted_tags = predict_tags(text, threshold=0.055, show_top_k=5, fallback=True)

    # 🧠 Convert predicted tags to binary vector
    pred_bin = np.zeros(len(mlb.classes_))
    for tag in predicted_tags:
        if tag in mlb.classes_:
            pred_bin[mlb.classes_.tolist().index(tag)] = 1

    # ✅ Append to final lists
    y_pred.append(pred_bin)
    y_true.append(val_labels[idx])  # already a binary array

# 🧾 Convert to NumPy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 📊 Evaluate using micro-averaged metrics
precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)

# ⏱️ Time taken
end = time.time()
elapsed = end - start

# ✅ Print results
print("\n✅ Evaluation Complete!")
print(f"🕒 Time Taken: {elapsed:.2f} seconds")
print(f"📌 Precision: {precision:.4f}")
print(f"📌 Recall:    {recall:.4f}")
print(f"📌 F1 Score:  {f1:.4f}")


🧪 Starting evaluation on 4863 validation samples...


🧠 Evaluating:   0%|          | 6/4863 [00:00<01:32, 52.33it/s]


🔍 Top 5 tag probabilities:
reactjs: 0.9139
tailwind-css: 0.4144
javascript: 0.2182
tanstack: 0.1383
css: 0.1209

🔍 Top 5 tag probabilities:
php: 0.8047
pdo: 0.7062
mysql: 0.4426
database: 0.0905
sql: 0.0869

🔍 Top 5 tag probabilities:
go: 0.9969
json: 0.0655
sql: 0.0387
postgresql: 0.0115
struct: 0.0107

🔍 Top 5 tag probabilities:
python: 0.8309
jupyter-notebook: 0.3176
jupyter: 0.2123
ipython: 0.0707
jupyter-lab: 0.0415

🔍 Top 5 tag probabilities:
python: 0.9493
pydantic: 0.6549
yaml: 0.1483
pydantic-v2: 0.0630
python-typing: 0.0473

🔍 Top 5 tag probabilities:
sql: 0.9598
sql-server: 0.4697
postgresql: 0.0834
t-sql: 0.0697
datetime: 0.0638

🔍 Top 5 tag probabilities:
scala: 0.9840
java: 0.0275
apache-spark: 0.0165
python: 0.0156
constructor: 0.0145

🔍 Top 5 tag probabilities:
c: 0.8272
pointers: 0.7647
c++: 0.2530
language-lawyer: 0.1492
function-pointers: 0.0643

🔍 Top 5 tag probabilities:
sql: 0.9137
sql-server: 0.0741
postgresql: 0.0565
database: 0.0552
oracle-database: 0.0487

🔍 

🧠 Evaluating:   0%|          | 18/4863 [00:00<01:47, 45.20it/s]


🔍 Top 5 tag probabilities:
kubernetes: 0.9774
kubectl: 0.0095
azure-aks: 0.0058
c#: 0.0029
google-kubernetes-engine: 0.0025

🔍 Top 5 tag probabilities:
node.js: 0.9861
express: 0.8125
javascript: 0.6635
mongodb: 0.5843
mongoose: 0.0977

🔍 Top 5 tag probabilities:
python: 0.8738
debugging: 0.1640
gdb: 0.0379
lldb: 0.0219
macos: 0.0159

🔍 Top 5 tag probabilities:
django: 0.8653
cron: 0.6704
python: 0.6025
windows: 0.0323
python-3.x: 0.0296

🔍 Top 5 tag probabilities:
swift: 0.8931
swiftui: 0.8306
ios: 0.7153
uikit: 0.0467
xcode: 0.0124

🔍 Top 5 tag probabilities:
c: 0.9126
pointers: 0.5755
memory: 0.0375
language-lawyer: 0.0329
arrays: 0.0250

🔍 Top 5 tag probabilities:
flutter: 0.9994
dart: 0.3568
flutter-layout: 0.1612
ios: 0.1363
flutter-dependencies: 0.0940

🔍 Top 5 tag probabilities:
swift: 0.3909
math: 0.0721
java: 0.0514
scenekit: 0.0433
unity-game-engine: 0.0372

🔍 Top 5 tag probabilities:
python: 0.9960
asynchronous: 0.6238
python-asyncio: 0.5923
python-3.x: 0.2765
async-await:

...

🧠 Evaluating: 100%|█████████▉| 4845/4863 [01:38<00:00, 61.72it/s]


🔍 Top 5 tag probabilities:
qt: 0.9834
c++: 0.6850
qt5: 0.2058
qml: 0.1852
qt6: 0.0580

🔍 Top 5 tag probabilities:
delphi: 0.9423
arrays: 0.0816
javascript: 0.0281
delphi-12-athens: 0.0266
firemonkey: 0.0250

🔍 Top 5 tag probabilities:
c: 0.1400
ocaml: 0.0594
algorithm: 0.0527
function: 0.0514
java: 0.0476

🔍 Top 5 tag probabilities:
azure: 0.8151
azure-blob-storage: 0.4972
azure-functions: 0.1801
azure-storage: 0.0572
c#: 0.0372

🔍 Top 5 tag probabilities:
reactjs: 0.8869
material-ui: 0.5752
typescript: 0.4553
mui-x-data-grid: 0.2347
javascript: 0.1066

🔍 Top 5 tag probabilities:
c#: 0.9944
.net: 0.2820
string: 0.2297
url: 0.0750
asp.net: 0.0734

🔍 Top 5 tag probabilities:
c: 0.9770
regex: 0.1653
c#: 0.0650
string: 0.0636
url: 0.0579

🔍 Top 5 tag probabilities:
android: 0.9570
video: 0.1384
kotlin: 0.1209
camera: 0.1087
webview: 0.0740

🔍 Top 5 tag probabilities:
css: 0.9734
svg: 0.9433
html: 0.5230
javascript: 0.1649
css-animations: 0.0616

🔍 Top 5 tag probabilities:
mui-x-data-grid:

🧠 Evaluating: 100%|██████████| 4863/4863 [01:39<00:00, 49.06it/s]


🔍 Top 5 tag probabilities:
postgresql: 0.9981
python: 0.9764
psycopg2: 0.2966
sqlalchemy: 0.1461
pandas: 0.0734

🔍 Top 5 tag probabilities:
cmake: 0.9882
c++: 0.5102
python: 0.0291
windows: 0.0225
linux: 0.0185

🔍 Top 5 tag probabilities:
python: 0.6272
windows: 0.5364
macos: 0.2645
command-line: 0.0641
python-3.x: 0.0493

🔍 Top 5 tag probabilities:
css: 0.9991
html: 0.9439
flexbox: 0.6673
media-queries: 0.0559
sass: 0.0461

🔍 Top 5 tag probabilities:
flutter: 0.9994
dart: 0.3406
android: 0.1506
ios: 0.1414
flutter-dependencies: 0.1117

🔍 Top 5 tag probabilities:
databricks: 0.9124
pyspark: 0.8667
apache-spark: 0.6810
apache-spark-sql: 0.3824
azure-databricks: 0.3760

🔍 Top 5 tag probabilities:
next.js: 0.9161
node.js: 0.9107
express: 0.7539
cookies: 0.3210
reactjs: 0.2555

🔍 Top 5 tag probabilities:
r: 0.9916
statistics: 0.1455
regression: 0.0557
random-forest: 0.0337
time-series: 0.0283

🔍 Top 5 tag probabilities:
hl7-fhir: 0.4649
python: 0.0339
.net: 0.0206
json: 0.0149
javascript:


✅ Evaluation Complete!
🕒 Time Taken: 101.13 seconds
📌 Precision: 0.4050
📌 Recall:    0.6628
📌 F1 Score:  0.5028


🚀 Starting training...
